In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from DataMatrix import generate_data_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

import warnings

warnings.filterwarnings("ignore")

In [ ]:
X_train, y_train, X_test, y_test = generate_data_matrix(method="mean")

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

### DBSCAN Implementation

In [ ]:
def get_DBSCAN(X, min_samples, eps):
    # Initialize labels
    labels = np.zeros(X.shape[0])

    # C is the cluster counter
    C = 0
    for i in range(X.shape[0]):

        # Skip if already labeled
        if labels[i] != 0:
            continue

        # Find neighbors within eps
        neighbors = np.where(np.linalg.norm(X - X[i], axis=1) <= eps)[0]

        # Mark as noise
        if len(neighbors) < min_samples:
            labels[i] = -1
            continue

        # New cluster
        C += 1

        # Assign cluster label to point
        labels[i] = C

        # Set of points to expand
        S = list(neighbors)

        i = 0
        while i < len(S):
            j = S[i]
            if labels[j] == -1:
                labels[j] = C
            elif labels[j] == 0:
                labels[j] = C
                neighbors_j = np.where(np.linalg.norm(X - X[j], axis=1) <= eps)[0]
                if len(neighbors_j) >= min_samples:
                    S += list(set(neighbors_j) - set(S))
            i += 1

    return labels

#### Labels Mapping Function with Ground Truth Labels

In [ ]:
def map_labels(y_true, y_pred):
    # Create a confusion matrix
    D = confusion_matrix(y_true, y_pred)

    # Swap the rows and columns of the confusion matrix to match the labels
    rows, cols = linear_sum_assignment(D, maximize=True)

    # Create a dictionary to map the labels
    label_map = {col: row for row, col in zip(rows, cols)}

    # Map the labels in y_pred, skipping over any points that have a label of -1
    y_pred = np.array([label_map[label] if label != -1 else -1 for label in y_pred])

    return y_pred

In [37]:
n_samples = 7
eps = 2.7

y_pred = get_DBSCAN(X_train, n_samples, eps)
y_pred_mapped = map_labels(y_train, y_pred)

# Compute the accuracy
precision = precision_score(y_train, y_pred_mapped, average="weighted")
recall = recall_score(y_train, y_pred_mapped, average="weighted")
f1 = f1_score(y_train, y_pred_mapped, average="weighted")

print("Precision: {:.3f}%".format(precision * 100))
print("Recall:    {:.3f}%".format(recall * 100))
print("F1-Score:  {:.3f}%".format(f1 * 100))

Precision: 84.479%
Recall:    30.099%
F1-Score:  35.468%


### DBSCAN in Scikit-Learn